## Part 1: Preprocessing

In [191]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [192]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [193]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [194]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'DistanceFromHome', 'Education', 'JobSatisfaction', 
                    'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 
                    'RelationshipSatisfaction', 'WorkLifeBalance', 'YearsAtCompany']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes

Age                          int64
DistanceFromHome             int64
Education                    int64
JobSatisfaction              int64
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
RelationshipSatisfaction     int64
WorkLifeBalance              int64
YearsAtCompany               int64
dtype: object

In [195]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [196]:
# Convert your X data to numeric data types however you see fit
X_df_numeric = pd.get_dummies(X_df, drop_first=True)


X_df_numeric.head()
X_df_numeric.dtypes


Age                         int64
DistanceFromHome            int64
Education                   int64
JobSatisfaction             int64
NumCompaniesWorked          int64
PercentSalaryHike           int64
RelationshipSatisfaction    int64
WorkLifeBalance             int64
YearsAtCompany              int64
OverTime_Yes                 bool
dtype: object

In [197]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_numeric = pd.get_dummies(X_train, drop_first=True)
X_test_numeric = pd.get_dummies(X_test, drop_first=True)

X_train_numeric, X_test_numeric = X_train_numeric.align(X_test_numeric, join='left', axis=1)
X_test_numeric = X_test_numeric.fillna(0)  

scaler.fit(X_train_numeric)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)


In [198]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False) 
# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = encoder.transform(y_train[['Department']])
department_test_encoded = encoder.transform(y_test[['Department']])

In [199]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False, drop='first')  

# Fit the encoder to the training data
encoder_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = encoder_attrition.transform(y_train[['Attrition']])
attrition_test_encoded = encoder_attrition.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [200]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(128, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(64, activation='relu')(shared_layer_1)

In [201]:
# Create a branch for Department with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation='relu')(shared_layer_2)

# Create the output layer
department_output_layer = layers.Dense(1, activation='softmax', name='department_output')(department_hidden_layer)


In [202]:
# Create a branch for Attrition with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output_layer = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)

In [203]:
# Create the model
from tensorflow.keras.models import Model

model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    optimizer='adam',
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 128)       │      1,408 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 64)        │      8,256 │ dense_36[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 32)        │      2,080 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_39 (Dense)    │ (None, 16)        │      1,040 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 1)         │         33 │ dense_38[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         17 │ dense_39[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,834 (50.13 KB)

 Trainable params: 12,834 (50.13 KB)

 Non-trainable params: 0 (0.00 B)

In [204]:
# Check the shapes of X_train_scaled and targets
print("X_train_scaled shape:", X_train_scaled.shape)
print("attrition_train_encoded shape:", attrition_train_encoded.shape)
print("department_train_encoded shape:", department_train_encoded.shape)

# Check the model summary for output layer details
model.summary()


X_train_scaled shape: (1176, 10)
attrition_train_encoded shape: (1176, 1)
department_train_encoded shape: (1176, 3)


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 128)       │      1,408 │ input_layer_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_37 (Dense)    │ (None, 64)        │      8,256 │ dense_36[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_38 (Dense)    │ (None, 32)        │      2,080 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_39 (Dense)    │ (None, 16)        │      1,040 │ dense_37[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 1)         │         33 │ dense_38[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         17 │ dense_39[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,834 (50.13 KB)

 Trainable params: 12,834 (50.13 KB)

 Non-trainable params: 0 (0.00 B)

In [205]:
# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    metrics={
        'department_output': ['accuracy'],  # Accuracy for department classification
        'attrition_output': ['accuracy']   # Accuracy for attrition classification
    }
)


In [206]:
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'binary_crossentropy'
    },
    metrics={
        'department_output': ['accuracy'],  # Metric for department classification
        'attrition_output': ['accuracy']   # Metric for attrition classification
    }
)

In [207]:
# Debug shapes
print("X_train_scaled shape:", X_train_scaled.shape) 
print("department_train_encoded shape:", department_train_encoded.shape) 
print("attrition_train_encoded shape:", attrition_train_encoded.shape)  

# Ensure unique values in targets
print("Unique values in attrition_train_encoded:", np.unique(attrition_train_encoded))
print("department_train_encoded first row:", department_train_encoded[0])


X_train_scaled shape: (1176, 10)
department_train_encoded shape: (1176, 3)
attrition_train_encoded shape: (1176, 1)
Unique values in attrition_train_encoded: [0. 1.]
department_train_encoded first row: [0. 1. 0.]


In [ ]:
# Train the model
history = model.fit(
    X_train_scaled,
    {
        'department_output': department_train_encoded,  # One-hot encoded categorical target
        'attrition_output': attrition_train_encoded  # Binary target
    },
    validation_data=(
        X_test_scaled,
        {
            'department_output': department_test_encoded,
            'attrition_output': attrition_test_encoded
        }
    ),
    epochs=100,  # Number of epochs
    batch_size=32  # Batch size
)


Epoch 1/100
 1/37 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step

ValueError: Cannot get result() since the metric has not yet been built.

In [209]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {
        'department_output': department_test_encoded,  # One-hot encoded categorical target
        'attrition_output': attrition_test_encoded_one_hot  # One-hot encoded binary target
    }
)

# Print evaluation results
print("Evaluation results:", evaluation)

c:\Users\Augus\anaconda3\envs\py310\lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 2), output.shape=(None, 1)

In [210]:
# Print the accuracy for both department and attrition
print(f"Department accuracy: {evaluation[3]:.4f}")  # Assuming department_output accuracy is at index 3
print(f"Attrition accuracy: {evaluation[4]:.4f}")   # Assuming attrition_output accuracy is at index 4

NameError: name 'evaluation' is not defined

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. **Is accuracy the best metric to use on this data? Why or why not?**
   Accuracy is a helpful metric, but it may not be the best depending on the data distribution. If the dataset is imbalanced (e.g., one department or attrition state dominates), accuracy can be misleading. Metrics like precision, recall, or F1-score might provide better insights into the model's performance, especially for underrepresented classes.

2. **What activation functions did you choose for your output layers, and why?**
   - **Department Output:** `softmax` was chosen because it is suitable for multi-class classification problems. It outputs probabilities for each class, ensuring they sum to 1, making it ideal for predicting the department.
   - **Attrition Output:** `sigmoid` and later `softmax`. The choice depends on whether the binary target is represented as a single value or one-hot encoded. `sigmoid` outputs probabilities for binary classification, while `softmax` handles multi-class or one-hot binary outputs.

3. **Can you name a few ways that this model might be improved?**
   - **Hyperparameter Tuning:** Experiment with different optimizers, learning rates, batch sizes, and epochs to find the optimal configuration.
   - **Regularization:** Add dropout layers or L2 regularization to reduce overfitting.
   - **Feature Engineering:** Include additional meaningful features or use feature selection to eliminate irrelevant ones.
   - **Data Augmentation:** Balance the dataset by oversampling underrepresented classes to improve the model's generalization.
   - **Model Complexity:** Experiment with deeper or wider architectures if the current model underfits the data.
   - **Cross-Validation:** Use k-fold cross-validation to evaluate model robustness.
